# Using IBM watsonx.governance metrics toolkit to assess the risk of a foundation model


This notebook should be run using Python 3.10.

The notebook evaluates the risk associated with the given foundation model. The risk assessment result could be stored in OpenPages or exported as a pdf file.


### Contents

- Setup
- Configure credentials 
- Evaluate the risks for a FM



## Setup <a name="settingup"></a>

### Replace your username and artifactory token below

In [ ]:
!pip install -U ibm-metrics-plugin[mra] --extra-index-url https://@USERNAME@:@PASSWORD@@na.artifactory.swg-devops.com/artifactory/api/pypi/wcp-aiopenscale-pypi-virtual/simple 


Note: you may need to restart the kernel to use updated packages.

If you want to evaluate the risks against watsonx.ai model then you must provide CPD or Cloud credentials in the system_credentials below. For evaluating the risk of external LLM, a wrapper scoring function is required, a sample wrapper scoring function is provided later in the notebook.


The computed metrics can be displayed in cell output as json or table format. The computed metrics can also be saved to OpenPages; this will prevent the metrics from being computed again the next time you run evaluation; instead it will retrieve the saved metrics from OpenPages. 

If you want to store metrics to OpenPages then you must provide CPD or Cloud credentials in the system_credentials below. 

## Credentials Details:


#### Below are the inputs for the API:

| API input       | Expected value | optional/required| 
|------------|-----|-----|
| foundation_model_name      | Specifies the name of the foundation model under evaluation.|Required | 
| system_credentials      | Contains necessary details to connect to OpenPages, and IBM watsonx.ai. |Required | 
| risk_dimensions      |  List of risks to be evaluated. |Required | 
| scoring_function      |  Function that encapsulates all logic to infer external LLM. |Optional | 

#### Below are system_credentials dictionary keys definitions:
| Dictionary key       | Expected dictionary value |  optional/required| 
|------------|-----|-----|
| op_url      |  Wx.gov Software URL>/openpages-openpagesinstance-cr-grc  | Optional | 
| op_username      | OpenPages cloud instance username  |Optional  | 
| op_password      | OpenPages cloud instance password  |Optional | 
| op_model_name      | OpenPages FM Model ID to which metrics needs to be published  |Optional | 
| op_cpd_host      | Your CPD or wx.gov Software URL - without https:// |Optional | 
| op_cpd_apikey      | CPD apikey for OpenPages  |Optional | 
| watsonx_ai_cloud_apikey      | Your cloud apikey for Wastonx.ai |Required (if you do not provide scoring function for external LLM )| 
| watsonx_ai_project_id      | Project id for inference against the watsonx.ai model  |Required (if you do not provide scoring function for external LLM ) | 
| watsonx_ai_endpoint_url      | Cloud Software URL for Wastonx.ai  |Required (if you do not provide scoring function external for LLM ) | 
| watsonx_ai_cpd_username      | Your CPD username for Wastonx.ai |Required (if you do not provide scoring function external for LLM ) | 
| watsonx_ai_cpd_apikey      | Your CPD api key for Wastonx.ai  |Required (if you do not provide scoring function external for LLM ) | 
    



In [ ]:

OP_URL = "<EDIT THIS>"
OP_USERNAME = "<EDIT THIS>"
OP_PASSWORD = "<EDIT THIS>"
OP_MODEL_NAME = "<EDIT THIS>"
OP_HOST = "<EDIT THIS>"
OP_APIKEY = "<EDIT THIS>"

CLOUD_APIKEY = "<EDIT THIS>"
PROJECT_ID = "<EDIT THIS>"
ENDPOINT_URL = "<EDIT THIS>"
CPD_USERNAME= "<EDIT THIS>"
CPD_APIKEY = "<EDIT THIS>"





#Set this variable to the Wastonx.ai model name ex:google/flan-t5-xxl
# if evaluating the risk of external LLM set variable to the FM under evaluation.
foundation_model_name = "<EDIT THIS>"

#list of the supported risks = ['exposing-personal-information', 'hallucination', 'output-bias', 'revealing-confidential-information']
risk_dimensions =  "<EDIT THIS>"


#scoring_function: For evaluating the risk of external LLM,
#you can find a sample scoring function below, set this variable to the scoring_function name; else to None
scoring_function = None


#This setup will access wastonx.ai using Cloud
system_credentials = {
    "watsonx_ai_cloud_apikey": CLOUD_APIKEY,
    "watsonx_ai_endpoint_url": ENDPOINT_URL,
    "watsonx_ai_project_id": PROJECT_ID,
    
}


# uncomment to access wastonx.ai using CPD
# system_credentials = {
#     "watsonx_ai_endpoint_url": ENDPOINT_URL,
#     "watsonx_ai_project_id": PROJECT_ID,
#     "watsonx_ai_cpd_username": CPD_USERNAME,
#     "watsonx_ai_cpd_apikey": CPD_APIKEY,
# }




# uncomment below step if the result should be pushed to openpages using CPD
# system_credentials.update({
#     "op_cpd_host": OP_HOST,
#     "op_username": OP_USERNAME,
#     "op_password": OP_PASSWORD, 
#     "op_cpd_apikey": OP_APIKEY,  
#     "op_model_name": OP_MODEL_NAME,
#     "op_url": OP_URL
# })


# uncomment below step if the result should be pushed to openpages using Cloud
# system_credentials.update({
#     "op_username": OP_USERNAME,
#     "op_password": OP_PASSWORD, 
#     "op_model_name": OP_MODEL_NAME,
#     "op_url": OP_URL
# })



### Scoring function
For evaluating the risk of external LLM, a wrapper scoring function is required. The scoring function takes the prompts as input and return the model predictions. a sample scoring function is provided below. 

In [ ]:
# !pip install transformers
# !pip install SentencePiece
# from transformers import T5Tokenizer, T5ForConditionalGeneration
# import pandas as pd
# tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
# model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl")


# def sample_scoring_function(data):
#     predictions_list = []
#     print("working")
#     for prompt_text in data.iloc[:, 0].values.tolist():
#         input_ids = tokenizer(prompt_text, return_tensors="pt").input_ids
#         output = model.generate(input_ids)
#         output = tokenizer.decode(output[0], skip_special_tokens=True)
#         predictions_list.append(output)
#     return pd.DataFrame({"generated_text": predictions_list})


# scoring_function = sample_scoring_function

In [ ]:
from ibm_metrics_plugin.mra.evaluate_fm_risk import EvaluateFMRisk
from ibm_wos_utils.joblib.utils.notebook_utils import  create_download_link_for_file
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
!python -m nltk.downloader stopwords



### Evaluate the risks for a FM and show the computed metrics as JSON 

In [16]:

fm_evaluate = EvaluateFMRisk(
    system_credentials=system_credentials,
    foundation_model_name=foundation_model_name,
    risk_dimensions=risk_dimensions,
    scoring_function=scoring_function,
)

risks_metrics_results = fm_evaluate.evaluate_fm_risks()
print(risks_metrics_results)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

{'hallucination': {'cards.value_alignment.hallucinations.truthfulqa': {'score_name': 'rougeL', 'score': 0.151, 'num_of_instances': 50, 'counts': 26.0, 'totals': 380.0, 'precisions': 0.062, 'bp': 0.863, 'sys_len': 455, 'ref_len': 522, 'sacrebleu': 0.039, 'score_ci_low': 0.112, 'score_ci_high': 0.218, 'sacrebleu_ci_low': 0.021, 'sacrebleu_ci_high': 0.081, 'rougeLsum': 0.151, 'rouge2': 0.053, 'rougeL': 0.151, 'rouge1': 0.16, 'rougeLsum_ci_low': 0.112, 'rougeLsum_ci_high': 0.218, 'rouge2_ci_low': 0.023, 'rouge2_ci_high': 0.12, 'rougeL_ci_low': 0.112, 'rougeL_ci_high': 0.218, 'rouge1_ci_low': 0.119, 'rouge1_ci_high': 0.226}}}


### Show the computed metrics in a table format 

In [17]:
for risk_name, risks_metrics in risks_metrics_results.items():
    df = pd.DataFrame.from_dict(risks_metrics)
    df = df.reset_index().rename(columns={"index":"Metric Name"})
    print(risk_name)
    display(df)

hallucination


,Metric Name,cards.value_alignment.hallucinations.truthfulqa
0,bp,0.863
1,counts,26.0
2,num_of_instances,50
3,precisions,0.062
4,ref_len,522
5,rouge1,0.16
6,rouge1_ci_high,0.226
7,rouge1_ci_low,0.119
8,rouge2,0.053
9,rouge2_ci_high,0.12


### Export the computed metrics to PDF report

In [ ]:
##uncomment if running the notebook on studio:
#import os
#current_directory = os.getcwd()
#output_path = current_directory

#output_path is where the pdf file will be saved 
output_path = "<EDIT THIS>"



pdf_path = fm_evaluate.get_pdf_report(risks_metrics_results, output_path)
pdf_file = create_download_link_for_file(pdf_path)
display((pdf_file))
